In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


import xgboost as xgb
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import glob, os, re
import mplhep as hep
import json

import bdtcode
import crosssections

from common import logger, DATADIR, Columns, time_and_log, imgcat, set_matplotlib_fontsizes, columns_to_numpy, ddt, rhoddt_windowcuts

/tmp/ipykernel_2298119/164350219.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
def is_array(a):
    """
    Checks if a thing is an array or maybe a number
    """
    try:
        shape = a.shape
        return len(shape) >= 1
    except AttributeError:
        return False

def calc_dphi(phi1, phi2):
    """
    Calculates delta phi. Assures output is within -pi .. pi.
    """
    twopi = 2.*np.pi
    # Map to 0..2pi range
    dphi = (phi1 - phi2) % twopi
    # Map pi..2pi --> -pi..0
    if is_array(dphi):
        dphi[dphi > np.pi] -= twopi
    elif dphi > np.pi:
        dphi -= twopi
    return dphi


def calculate_mass(pt, eta, e):
    """
    compute mass
    """
    pz = pt * np.sinh(eta)
    mass = np.sqrt(e**2 - pt**2 - pz**2)
    return mass

def calculate_rho(pt, eta, e):
    mass = calculate_mass(pt, eta, e)
    rho  = np.log(mass**2 / pt**2)
    return rho

In [3]:
sigbase_path = '/home/snabili/data/svj_local_scripts/rho_bdt_allfiles/'
bkgbase_path = '/home/snabili/hadoop/BKG/Ultra_Legacy/HADD_BKGBDT/'

sig_path    = sigbase_path + 'signal_notruth/madpt300_mz'
bkg_path    = bkgbase_path + 'Summer20UL18/'

qcd_dirs    = [f for f in os.listdir(bkg_path) if re.search('qcd*',    f, flags=re.IGNORECASE)]
ttjets_dirs = [f for f in os.listdir(bkg_path) if re.search('ttjets*', f, flags=re.IGNORECASE)]
wjets_dirs  = [f for f in os.listdir(bkg_path) if re.search('wjets*',  f, flags=re.IGNORECASE)]
zjets_dirs  = [f for f in os.listdir(bkg_path) if re.search('zjets*',  f, flags=re.IGNORECASE)]
 
qcd_length    = len(qcd_dirs)
ttjets_length = len(ttjets_dirs)
wjets_length  = len(wjets_dirs)
zjets_length  = len(zjets_dirs)

qcd_files    = [np.load(bkg_path+qcd_dirs[i],    allow_pickle=True,encoding='bytes') for i in range(qcd_length)]
ttjets_files = [np.load(bkg_path+ttjets_dirs[i], allow_pickle=True,encoding='bytes') for i in range(ttjets_length)]
wjets_files  = [np.load(bkg_path+wjets_dirs[i],  allow_pickle=True,encoding='bytes') for i in range(wjets_length)]
zjets_files  = [np.load(bkg_path+zjets_dirs[i],  allow_pickle=True,encoding='bytes') for i in range(zjets_length)]

sig_files_rinv  = [np.load(sig_path+str(i)+'_mdark10_rinv'+ str(k) + '.npz',         allow_pickle=True,encoding='bytes') for i in np.arange(200, 555,50) for k in (0.1,0.3,0.7)]
sig_files_mdark = [np.load(sig_path+str(i)+'_mdark'       + str(j) + '_rinv0.3.npz', allow_pickle=True,encoding='bytes') for i in np.arange(200, 555,50) for j in (1, 5, 10)]

sig_length_rinv  = len(sig_files_rinv)
sig_length_mdark  = len(sig_files_mdark)

qcd_xsec       = np.array([crosssections.get_xs(i) for i in qcd_dirs])
ttjets_xsec    = np.array([crosssections.get_xs(i) for i in ttjets_dirs])
wjets_xsec     = np.array([crosssections.get_xs(i) for i in wjets_dirs])
zjets_xsec     = np.array([crosssections.get_xs(i) for i in zjets_dirs])
sig_xsec       = np.array([118.6, 118.6, 118.6, 112.7, 112.7, 112.7, 108.5, 108.5, 108.5, 98.53, 98.53, 98.53, 89.05, 89.05, 89.05, 80.33, 80.33, 80.33, 71.02, 71.02, 71.02, 60.72, 60.72, 60.72])


run2018_prehem = 14026.948 + 7044.413

qcd_sel_eff            = np.array([qcd_files[n]['cutflow_vals'][11]/qcd_files[n]['cutflow_vals'][0] for n in range(qcd_length)])
ttjets_sel_eff         = np.array([ttjets_files[n]['cutflow_vals'][11]/ttjets_files[n]['cutflow_vals'][0] for n in range(ttjets_length)])
wjets_sel_eff          = np.array([wjets_files[n]['cutflow_vals'][11]/wjets_files[n]['cutflow_vals'][0] for n in range(wjets_length)])
zjets_sel_eff          = np.array([zjets_files[n]['cutflow_vals'][11]/zjets_files[n]['cutflow_vals'][0] for n in range(zjets_length)])

sig_sel_eff_rinv       = np.array([sig_files_rinv[n]['cutflow_vals'][11]/sig_files_rinv[n]['cutflow_vals'][0] for n in range(sig_length_rinv)])
sig_sel_eff_mdark      = np.array([sig_files_mdark[n]['cutflow_vals'][11]/sig_files_mdark[n]['cutflow_vals'][0] for n in range(sig_length_mdark)])

qcd_pass               = np.array([qcd_files[n]['cutflow_vals'][11] for n in range(qcd_length)])
ttjets_pass            = np.array([ttjets_files[n]['cutflow_vals'][11] for n in range(ttjets_length)])
wjets_pass             = np.array([wjets_files[n]['cutflow_vals'][11] for n in range(wjets_length)])
zjets_pass             = np.array([zjets_files[n]['cutflow_vals'][11] for n in range(zjets_length)])

sig_pass_rinv          = np.array([sig_files_rinv[n]['cutflow_vals'][11] for n in range(sig_length_rinv)])
sig_pass_mdark         = np.array([sig_files_mdark[n]['cutflow_vals'][11] for n in range(sig_length_mdark)])


qcd_weight_run2018_prehem          = qcd_xsec         * qcd_sel_eff       * run2018_prehem / qcd_pass
ttjets_weight_run2018_prehem       = ttjets_xsec         * ttjets_sel_eff       * run2018_prehem / ttjets_pass
wjets_weight_run2018_prehem        = wjets_xsec         * wjets_sel_eff       * run2018_prehem / wjets_pass
zjets_weight_run2018_prehem        = zjets_xsec         * zjets_sel_eff       * run2018_prehem / zjets_pass

gq = 0.25
sig_weight_run2018_prehem_rinv  = sig_xsec * gq**2 * sig_sel_eff_rinv  * run2018_prehem / sig_pass_rinv 
sig_weight_run2018_prehem_mdark = sig_xsec * gq**2 * sig_sel_eff_mdark * run2018_prehem / sig_pass_mdark


TTJets_DiLept_TuneCP5_13TeV-madgraphMLM-pythia8.npz: Found BR: 831.8 x 0.105 = 87.339 <-- eff xs
Key TTJets_DiLept_genMET-150 matched but could not retrieve a cross section
TTJets_DiLept_genMET-150_TuneCP5_13TeV-madgraphMLM-pythia8.npz: Found BR: 831.8 x 0.105 = 87.339 <-- eff xs
TTJets_SingleLeptFromT_TuneCP5_13TeV-madgraphMLM-pythia8.npz: Found BR: 831.8 x 0.219 = 182.1642 <-- eff xs
Key TTJets_SingleLeptFromT_genMET-150 matched but could not retrieve a cross section
TTJets_SingleLeptFromT_genMET-150_TuneCP5_13TeV-madgraphMLM-pythia8.npz: Found BR: 831.8 x 0.219 = 182.1642 <-- eff xs
TTJets_SingleLeptFromTbar_TuneCP5_13TeV-madgraphMLM-pythia8.npz: Found BR: 831.8 x 0.219 = 182.1642 <-- eff xs
Key TTJets_SingleLeptFromTbar_genMET-150 matched but could not retrieve a cross section
TTJets_SingleLeptFromTbar_genMET-150_TuneCP5_13TeV-madgraphMLM-pythia8.npz: Found BR: 831.8 x 0.219 = 182.1642 <-- eff xs


In [4]:
qcd_stacked    = {}
ttjets_stacked = {}
wjets_stacked  = {}
zjets_stacked  = {}
sig_rinv={}
sig_mdark={}
for i in ('pt','eta','e', 'girth','mt','metdphi', 'met','mass','rho','girthddt', 'phi', 'rt'):

    qcd_stacked[i]    = np.hstack(list(qcd_files[n]['arrays'].tolist()[i] for n in range(qcd_length)))
    ttjets_stacked[i] = np.hstack(list(ttjets_files[n]['arrays'].tolist()[i] for n in range(ttjets_length)))
    wjets_stacked[i]  = np.hstack(list(wjets_files[n]['arrays'].tolist()[i] for n in range(wjets_length)))
    zjets_stacked[i]  = np.hstack(list(zjets_files[n]['arrays'].tolist()[i] for n in range(zjets_length)))
    
    sig_rinv[i] = [sig_files_rinv[n]['arrays'].tolist()[i] for n in range(sig_length_rinv)]
    sig_mdark[i] = [sig_files_mdark[n]['arrays'].tolist()[i] for n in range(sig_length_mdark)]
    
qcd_weights_stacked = np.hstack(list(np.ones_like(qcd_files[n]['arrays'].tolist()['mt'])*qcd_weight_run2018_prehem[n] for n in range(qcd_length)))
ttjets_weights_stacked = np.hstack(list(np.ones_like(ttjets_files[n]['arrays'].tolist()['mt'])*ttjets_weight_run2018_prehem[n] for n in range(ttjets_length)))
wjets_weights_stacked = np.hstack(list(np.ones_like(wjets_files[n]['arrays'].tolist()['mt'])*wjets_weight_run2018_prehem[n] for n in range(wjets_length)))
zjets_weights_stacked = np.hstack(list(np.ones_like(zjets_files[n]['arrays'].tolist()['mt'])*zjets_weight_run2018_prehem[n] for n in range(zjets_length)))

sig_rinv_weights = list(np.ones_like(sig_files_rinv[n]['arrays'].tolist()['mt'])*sig_weight_run2018_prehem_rinv[n] for n in range(sig_length_rinv))
sig_mdark_weights = list(np.ones_like(sig_files_mdark[n]['arrays'].tolist()['mt'])*sig_weight_run2018_prehem_mdark[n] for n in range(sig_length_mdark))



In [5]:
from scipy.ndimage import gaussian_filter

def rhoddt_windowcuts(mt, pt, rho):
    cuts = (mt>200) & (mt<1000) & (pt>110) & (pt<1500) & (rho>-4) & (rho<0)
    return cuts

def varmap(mt, pt, rho, var, weight, percent):    
    cuts = rhoddt_windowcuts(mt, pt, rho)
    C, RHO_edges, PT_edges = np.histogram2d(rho[cuts], pt[cuts], bins=14,weights=weight[cuts])
    w, h = 15, 15
    VAR_map      = [[0 for x in range(w)] for y in range(h)]
    VAR = var[cuts]
    for i in range(len(RHO_edges)-1):
        for j in range(len(PT_edges)-1):
            CUT = (rho[cuts]>RHO_edges[i]) & (rho[cuts]<RHO_edges[i+1]) & (pt[cuts]>PT_edges[j]) & (pt[cuts]<PT_edges[j+1])
            if len(VAR[CUT])==0: continue
            if len(VAR[CUT])>0:
                VAR_map[i][j]=np.percentile(VAR[CUT],(100-percent))

    VAR_map_smooth = gaussian_filter(VAR_map,1)
    return VAR_map_smooth, RHO_edges, PT_edges




def ddt(mt, pt, rho, var, weight, percent):
    with time_and_log(f'Calculating girth ddt for ...{percent}'):
        cuts = rhoddt_windowcuts(mt, pt, rho)
        var_map_smooth, RHO_edges, PT_edges = varmap(mt, pt, rho, var, weight, percent)
        nbins = 14
        Pt_min, Pt_max = min(PT_edges), max(PT_edges)
        Rho_min, Rho_max = min(RHO_edges), max(RHO_edges)

        ptbin_float  = nbins*(pt-Pt_min)/(Pt_max-Pt_min)
        rhobin_float = nbins*(rho-Rho_min)/(Rho_max-Rho_min)
        '''ptbin  = np.clip(1 + np.round(ptbin_float).astype(int), 0, nbins)
        rhobin = np.clip(1 + np.round(rhobin_float).astype(int), 0, nbins)'''
        ptbin  = np.clip(1 + ptbin_float.astype(int), 0, nbins)
        rhobin = np.clip(1 + rhobin_float.astype(int),0, nbins)


        varDDT = np.array([var[i] - var_map_smooth[rhobin[i]-1][ptbin[i]-1] for i in range(len(var))])
        #return varDDT, rhobin, ptbin, var_map_smooth, RHO_edges, PT_edges
        return varDDT

In [6]:
qcd_stacked['girthDDT'] = ddt(qcd_stacked['mt'], qcd_stacked['pt'], qcd_stacked['rho'], qcd_stacked['girth'], qcd_weights_stacked, 47.8)
ttjets_stacked['girthDDT'] = ddt(ttjets_stacked['mt'], ttjets_stacked['pt'], ttjets_stacked['rho'], ttjets_stacked['girth'], ttjets_weights_stacked, 47.8)
wjets_stacked['girthDDT'] = ddt(wjets_stacked['mt'], wjets_stacked['pt'], wjets_stacked['rho'], wjets_stacked['girth'], wjets_weights_stacked, 47.8)
zjets_stacked['girthDDT'] = ddt(zjets_stacked['mt'], zjets_stacked['pt'], zjets_stacked['rho'], zjets_stacked['girth'], zjets_weights_stacked, 47.8)
sig_rinv['girthDDT'] = [ddt(sig_rinv['mt'][n], sig_rinv['pt'][n], sig_rinv['rho'][n], sig_rinv['girth'][n], sig_rinv_weights[n], 47.8) for n in range(sig_length_rinv)]
sig_mdark['girthDDT'] = [ddt(sig_mdark['mt'][n], sig_mdark['pt'][n], sig_mdark['rho'][n], sig_mdark['girth'][n], sig_mdark_weights[n], 47.8) for n in range(sig_length_mdark)]


[bdt:INFO:2023-05-16 14:05:17:common:40 hepcms-in1.umd.edu] Calculating girth ddt for ...47.8
[bdt:INFO:2023-05-16 14:06:36:common:47 hepcms-in1.umd.edu] Done (took 01m:18.73s)
[bdt:INFO:2023-05-16 14:06:36:common:40 hepcms-in1.umd.edu] Calculating girth ddt for ...47.8
[bdt:INFO:2023-05-16 14:07:01:common:47 hepcms-in1.umd.edu] Done (took 00m:24.97s)
[bdt:INFO:2023-05-16 14:07:01:common:40 hepcms-in1.umd.edu] Calculating girth ddt for ...47.8
[bdt:INFO:2023-05-16 14:07:16:common:47 hepcms-in1.umd.edu] Done (took 00m:15.69s)
[bdt:INFO:2023-05-16 14:07:16:common:40 hepcms-in1.umd.edu] Calculating girth ddt for ...47.8
[bdt:INFO:2023-05-16 14:07:25:common:47 hepcms-in1.umd.edu] Done (took 00m:8.48s)
[bdt:INFO:2023-05-16 14:07:25:common:40 hepcms-in1.umd.edu] Calculating girth ddt for ...47.8
[bdt:INFO:2023-05-16 14:07:25:common:47 hepcms-in1.umd.edu] Done (took 00m:0.28s)
[bdt:INFO:2023-05-16 14:07:25:common:40 hepcms-in1.umd.edu] Calculating girth ddt for ...47.8
[bdt:INFO:2023-05-16 14

[bdt:INFO:2023-05-16 14:07:33:common:47 hepcms-in1.umd.edu] Done (took 00m:0.13s)
[bdt:INFO:2023-05-16 14:07:33:common:40 hepcms-in1.umd.edu] Calculating girth ddt for ...47.8
[bdt:INFO:2023-05-16 14:07:33:common:47 hepcms-in1.umd.edu] Done (took 00m:0.25s)
[bdt:INFO:2023-05-16 14:07:33:common:40 hepcms-in1.umd.edu] Calculating girth ddt for ...47.8
[bdt:INFO:2023-05-16 14:07:33:common:47 hepcms-in1.umd.edu] Done (took 00m:0.24s)
[bdt:INFO:2023-05-16 14:07:33:common:40 hepcms-in1.umd.edu] Calculating girth ddt for ...47.8
[bdt:INFO:2023-05-16 14:07:34:common:47 hepcms-in1.umd.edu] Done (took 00m:0.20s)
[bdt:INFO:2023-05-16 14:07:34:common:40 hepcms-in1.umd.edu] Calculating girth ddt for ...47.8
[bdt:INFO:2023-05-16 14:07:34:common:47 hepcms-in1.umd.edu] Done (took 00m:0.32s)
[bdt:INFO:2023-05-16 14:07:34:common:40 hepcms-in1.umd.edu] Calculating girth ddt for ...47.8
[bdt:INFO:2023-05-16 14:07:34:common:47 hepcms-in1.umd.edu] Done (took 00m:0.38s)
[bdt:INFO:2023-05-16 14:07:34:common:4

In [7]:
mz    = np.arange(200,560,50)
rinv  = [0.1, 0.3, 0.7]
mdark = [1,   5,  10]

sig_label_mdark = ['BSVJ_'+str(i)+ '_'    +str(j)+ '_0.3_peak' for i in mz for j in mdark]
sig_label_rinv  = ['BSVJ_'+str(i)+ '_10_' +str(k)+ '_peak'     for i in mz for k in rinv]

In [10]:
data_path          = '/home/snabili/hadoop/Data/Run2_03302023/'
data_dir = {}
data_dir['run2018_prehem'] = [f for f in os.listdir(data_path) if f.startswith('Run2018A') or f.startswith('Run2018B')]
data_dir['run2018_posthem'] = [f for f in os.listdir(data_path) if f.startswith('Run2018C') or f.startswith('Run2018D')]

data = {}
data_cut = {}
data_files = {}
for i in ('run2018_prehem', 'run2018_posthem'): 
    #data[i]['mass'] = calculate_mass(pt, eta, e)
    data[i]={}       
    data_files[i] = [np.load(data_path+data_dir[i][n]) for n in range(len(data_dir[i]))]
    
    data_cut[i] = [(abs(data_files[i][n]['eta'])<2.4) & (data_files[i][n]['rt']>1.1) & (data_files[i][n]['ak8_lead_pt']>500) & (data_files[i][n]['ecfc2b1']>0) & (abs(calc_dphi(data_files[i][n]['metphi'], data_files[i][n]['phi']))<1.5) for n in range(len(data_dir[i]))]
    
    data[i]['metdphi'] = np.hstack(list(calc_dphi(data_files[i][k]['metphi'][data_cut[i][k]], data_files[i][k]['phi'][data_cut[i][k]]) for k in range(len(data_files[i])))) 
    data[i]['mass'] = np.hstack(list(calculate_mass(data_files[i][k]['pt'][data_cut[i][k]],data_files[i][k]['eta'][data_cut[i][k]], data_files[i][k]['e'][data_cut[i][k]]) for k in range(len(data_files[i]))))
    data[i]['rho'] = np.hstack(list(calculate_rho(data_files[i][k]['pt'][data_cut[i][k]],data_files[i][k]['eta'][data_cut[i][k]], data_files[i][k]['e'][data_cut[i][k]]) for k in range(len(data_files[i]))))
    #data[i]['girthDDT'] = np.hstack(list(ddt(data_files[i][k]['mt'], data_files[i][k]['pt'], data_files[i][k]['rho'], data_files[i][k]['girth'], np.ones_like(data_files[i][k]['mt']), 47.8) for k in range(len(data_files[i]))))
    
    for n in ('met', 'pt', 'rt', 'leading_pt', 'mt', 'girth', 'ak8_lead_pt', 'ecfc2b1', 'ak4_subl_eta', 'ak4_subl_phi', 'leading_eta', 'metphi', 'phi', 'eta', 'girth'):
        data[i][n] = np.hstack(list(data_files[i][k][n][data_cut[i][k]] for k in range(len(data_files[i]))))
    data[i]['girthDDT'] = ddt(data[i]['mt'], data[i]['pt'], data[i]['rho'], data[i]['girth'], np.ones_like(data[i]['rho']), 47.8)
    
        

[bdt:INFO:2023-05-16 14:08:32:common:40 hepcms-in1.umd.edu] Calculating girth ddt for ...47.8
[bdt:INFO:2023-05-16 14:08:54:common:47 hepcms-in1.umd.edu] Done (took 00m:21.94s)
[bdt:INFO:2023-05-16 14:10:18:common:40 hepcms-in1.umd.edu] Calculating girth ddt for ...47.8
[bdt:INFO:2023-05-16 14:11:05:common:47 hepcms-in1.umd.edu] Done (took 00m:46.94s)


In [80]:
def Ratio(bkg,data,weight, data_min, data_max): # to plot ratio plots
    
    # input to ratio function bkg = [z,w,t,q], weight = [z_w, w_w, t_w, q_w], data = d
    b=np.linspace(data_min, data_max,50)
    #b[var]=np.linspace(data.min(), var_max,50)
    val_of_bins_bkg,  edges_of_bins_bkg, patches_bkg = plt.hist(bkg, bins=b, weights=weight, stacked=True)
    val_of_bins_data, edges_of_bins_data, patches_data = plt.hist(data, bins=b)
    
    ratio = np.divide(val_of_bins_data, val_of_bins_bkg[3], where=((val_of_bins_bkg[3] != 0) & (val_of_bins_data>15)))
    error = np.divide(val_of_bins_data * np.sqrt(val_of_bins_bkg[3]) + val_of_bins_bkg[3] * np.sqrt(val_of_bins_data),
                    np.power(val_of_bins_bkg[3], 2),
                    where=((val_of_bins_bkg[3] != 0) & (val_of_bins_data>15)))  
    bincenter = 0.5 * (edges_of_bins_data[1:][val_of_bins_data>15] + edges_of_bins_data[:-1][val_of_bins_data>15])
    plt.close()
    return bincenter, ratio, error, val_of_bins_data


var = ['met', 'mt', 'pt','eta','mass', 'metdphi', 'phi', 'rt']
l_max = [1500, 1000,1100,2.4, 1100, 1.5, 3.14, 2.5]
l_min = [0, 0,0,0, 0, 0, -3.14, 1]


bins={} 
ratio={}
val_of_bins_data={}
error={}
k=0
k_fact=1
for l in var:
    print(l)
    bins[l] = {}
    ratio[l] = {}
    error[l] = {}
    val_of_bins_data[l] = {}
    #for k in (1.1,1.2,1.3,1.4,1.5,1.6,1.7,1.8):
    bins[l][1.5], ratio[l][1.5], error[l][1.5], val_of_bins_data[l][1.5] = Ratio([zjets_stacked[l],wjets_stacked[l],ttjets_stacked[l],qcd_stacked[l]], data['run2018_prehem'][l], 
                            weight=[zjets_weights_stacked, wjets_weights_stacked, ttjets_weights_stacked, k_fact*qcd_weights_stacked], data_min=l_min[k], data_max=l_max[k])
    k+=1

met
mt
pt
eta
mass
metdphi
phi
rt


In [82]:
g_cut = -0.1
q_cuts = (qcd_stacked['girthDDT']<g_cut)
t_cuts = (ttjets_stacked['girthDDT']<g_cut)
w_cuts = (wjets_stacked['girthDDT']<g_cut)
z_cuts = (zjets_stacked['girthDDT']<g_cut)
d_cuts = (data['run2018_prehem']['girthDDT']<g_cut)
b={}
#b=np.linspace(data['run2018_prehem']['met'][d_cuts].min(), data['run2018_prehem']['met'][d_cuts].max(),50)
b['met']=np.linspace(data['run2018_prehem']['met'][d_cuts].min(), 1500,50)
b['mt']=np.linspace(data['run2018_prehem']['mt'][d_cuts].min(), 1000,50)
b['pt']=np.linspace(data['run2018_prehem']['met'][d_cuts].min(), 1100,50)
b['mass']=np.linspace(data['run2018_prehem']['mass'][d_cuts].min(), 1000,50)
b['metdphi']=np.linspace(0, 1.5,50)
b['phi']=np.linspace(data['run2018_prehem']['phi'][d_cuts].min(), data['run2018_prehem']['phi'][d_cuts].max(),50)
b['eta']=np.linspace(0, data['run2018_prehem']['eta'][d_cuts].max(),50)
b['rt']=np.linspace(1, 2.5,50)

k_fact=1


for i in ('met', 'mt', 'pt','mass', 'metdphi', 'phi', 'eta', 'rt'):
    #fig.add_subplot(r, c, g)
    fig, (a0, a1) = plt.subplots(2, 1, gridspec_kw={'height_ratios': [3, 1]}, sharex=True, figsize=(10,7))
    fig.subplots_adjust(hspace=0)
    hep.cms.label(data=True, ax=a0, lumi=21.071, fontsize=20)

    l2=a0.hist([zjets_stacked[i][z_cuts], wjets_stacked[i][w_cuts], ttjets_stacked[i][t_cuts], qcd_stacked[i][q_cuts]], bins=b[i], 
               weights=[zjets_weights_stacked[z_cuts], wjets_weights_stacked[w_cuts],ttjets_weights_stacked[t_cuts],k_fact*qcd_weights_stacked[q_cuts]], label=['zjets', 'wjets', 'ttjets', 'qcd'], stacked=True)

    l1=a0.hist(data['run2018_prehem'][i][d_cuts], bins=b[i], histtype='step', linewidth=3, label='data')
    a0.axhline(y=15, color='black')
    a0.legend()
    a0.set_yscale('log')
    a0.set_ylabel('A.U.', fontsize=20)
    a0.set_title('$k_{factor}$='+str(k_fact), y=0.85, fontsize=15)
    a1.errorbar(bins[i][1.5], ratio[i][1.5][val_of_bins_data[i][1.5]>15], yerr=error[i][1.5][val_of_bins_data[i][1.5]>15], fmt='.', color='r')
    a1.axhline(y=1, linestyle = '--')
    a1.yaxis.set_ticks(np.arange(0, 1.8, 0.4))
    a1.set_ylim(0,2)
    a1.set_xlabel(str(i), fontsize=15)
    a1.set_ylabel('Data/MC', fontsize=15)
    #g+=1
    plt.savefig('ration_nokfactor_'+str(i)+'.png')
    plt.close()
